# SageMaker Modell zu ONNX konvertieren

Dieses Notebook konvertiert das SageMaker-Modell (PyTorch Lightning Checkpoint) zu ONNX.

## Schritte:
1. Installation der benötigten Pakete
2. Laden des Checkpoints und der Konfiguration
3. Erstellen des Modells und Laden der Gewichte
4. Konvertierung zu ONNX
5. Validierung des ONNX-Modells


In [13]:
# Installation der benötigten Pakete
%pip install -q transformers torch optimum[onnxruntime] onnxruntime


Note: you may need to restart the kernel to use updated packages.


In [14]:
# Imports
import torch
import json
import numpy as np
from pathlib import Path
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from transformers.onnx import export
import onnxruntime as ort
import warnings
warnings.filterwarnings('ignore')

print("✅ Alle Pakete erfolgreich importiert")


✅ Alle Pakete erfolgreich importiert


## 1. Pfade definieren


In [15]:
# Pfade
SAGEMAKER_MODEL_PATH = Path("../modelsagemaker")
SAGEMAKER_CKPT_PATH = SAGEMAKER_MODEL_PATH / "model.ckpt"
SAGEMAKER_HF_TEXT_PATH = SAGEMAKER_MODEL_PATH / "hf_text"
SAGEMAKER_ONNX_PATH = SAGEMAKER_MODEL_PATH / "model.onnx"
ASSETS_JSON_PATH = SAGEMAKER_MODEL_PATH / "assets.json"

print("Pfade:")
print(f"  Checkpoint: {SAGEMAKER_CKPT_PATH}")
print(f"  Konfiguration: {SAGEMAKER_HF_TEXT_PATH}")
print(f"  ONNX-Ausgabe: {SAGEMAKER_ONNX_PATH}")

# Prüfe ob Dateien existieren
if not SAGEMAKER_CKPT_PATH.exists():
    raise FileNotFoundError(f"Checkpoint nicht gefunden: {SAGEMAKER_CKPT_PATH}")
if not SAGEMAKER_HF_TEXT_PATH.exists():
    raise FileNotFoundError(f"Konfiguration nicht gefunden: {SAGEMAKER_HF_TEXT_PATH}")

print("\n✅ Alle benötigten Dateien gefunden")


Pfade:
  Checkpoint: ..\modelsagemaker\model.ckpt
  Konfiguration: ..\modelsagemaker\hf_text
  ONNX-Ausgabe: ..\modelsagemaker\model.onnx

✅ Alle benötigten Dateien gefunden


## 2. Konfiguration und Anzahl Labels laden


In [16]:
# Lade Konfiguration
print("Lade Konfiguration...")
config = AutoConfig.from_pretrained(str(SAGEMAKER_HF_TEXT_PATH))
print(f"✅ Konfiguration geladen: {config.model_type}")

# Bestimme Anzahl Labels
num_labels = 2  # Standard für Hate Speech Detection
if ASSETS_JSON_PATH.exists():
    with open(ASSETS_JSON_PATH, 'r') as f:
        assets = json.load(f)
        if 'output_shape' in assets and assets['output_shape']:
            num_labels = assets['output_shape']
            print(f"Anzahl Labels aus assets.json: {num_labels}")
        else:
            print(f"Verwende Standard: {num_labels} Labels")
else:
    print(f"assets.json nicht gefunden, verwende Standard: {num_labels} Labels")

# Setze num_labels in der Konfiguration
if not hasattr(config, 'num_labels') or config.num_labels is None:
    config.num_labels = num_labels
else:
    # Verwende die Anzahl aus der Konfiguration, aber prüfe Konsistenz
    if config.num_labels != num_labels:
        print(f"⚠️  Warnung: config.num_labels ({config.num_labels}) != assets.output_shape ({num_labels})")
        print(f"Verwende: {num_labels} (aus assets.json)")
        config.num_labels = num_labels

print(f"\n✅ Anzahl Labels: {config.num_labels}")


Lade Konfiguration...
✅ Konfiguration geladen: deberta-v2
Anzahl Labels aus assets.json: 3
⚠️  Warnung: config.num_labels (2) != assets.output_shape (3)
Verwende: 3 (aus assets.json)

✅ Anzahl Labels: 3


## 3. Modell erstellen


In [17]:
# Erstelle Modell mit der Konfiguration
print(f"Erstelle Modell mit {config.num_labels} Labels...")
model = AutoModelForSequenceClassification.from_config(config)
print(f"✅ Modell erstellt")
print(f"   Modell-Typ: {type(model).__name__}")
print(f"   Anzahl Parameter: {sum(p.numel() for p in model.parameters()):,}")


Erstelle Modell mit 3 Labels...
✅ Modell erstellt
   Modell-Typ: DebertaV2ForSequenceClassification
   Anzahl Parameter: 184,424,451


## 4. Gewichte aus Checkpoint laden


In [18]:
# Lade Checkpoint
print(f"Lade Checkpoint: {SAGEMAKER_CKPT_PATH}")
print(f"Dateigröße: {SAGEMAKER_CKPT_PATH.stat().st_size / (1024**3):.2f} GB")

checkpoint = torch.load(str(SAGEMAKER_CKPT_PATH), map_location='cpu')

# Prüfe Checkpoint-Struktur
print(f"\nCheckpoint-Struktur:")
if isinstance(checkpoint, dict):
    print(f"  Keys: {list(checkpoint.keys())[:10]}...")
    
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
        print(f"  state_dict gefunden mit {len(state_dict)} Keys")
    else:
        state_dict = checkpoint
        print(f"  Direktes state_dict mit {len(state_dict)} Keys")
else:
    state_dict = checkpoint
    print(f"  Checkpoint ist direkt state_dict mit {len(state_dict)} Keys")

# Zeige erste Keys
first_keys = list(state_dict.keys())[:5]
print(f"\nErste Keys (Beispiele):")
for key in first_keys:
    print(f"  {key}")


Lade Checkpoint: ..\modelsagemaker\model.ckpt
Dateigröße: 0.68 GB

Checkpoint-Struktur:
  Keys: ['state_dict']...
  state_dict gefunden mit 201 Keys

Erste Keys (Beispiele):
  model.model.embeddings.position_ids
  model.model.embeddings.word_embeddings.weight
  model.model.embeddings.LayerNorm.weight
  model.model.embeddings.LayerNorm.bias
  model.model.encoder.layer.0.attention.self.query_proj.weight


In [19]:
# Bereinige state_dict: Entferne Präfixe und mappe auf Deberta-Struktur
print("\nBereinige state_dict...")
cleaned_state_dict = {}

for key, value in state_dict.items():
    new_key = key
    
    # Entferne 'position_ids' (wird automatisch erstellt)
    if 'position_ids' in new_key:
        continue
    
    # Schritt 1: Entferne doppelte Präfixe (model.model. -> model.)
    if new_key.startswith('model.model.'):
        new_key = new_key[12:]  # Entferne 'model.model.' (12 Zeichen)
    elif new_key.startswith('model.'):
        new_key = new_key[6:]  # Entferne 'model.' (6 Zeichen)
    
    # Schritt 2: Entferne weitere Präfixe
    for prefix in ['module.', 'backbone.']:
        if new_key.startswith(prefix):
            new_key = new_key[len(prefix):]
            break
    
    # Schritt 3: WICHTIG - Für DebertaV2ForSequenceClassification muss 'deberta.' Präfix hinzugefügt werden
    # für Embedding/Encoder Keys, aber NICHT für classifier/pooler/head
    if not new_key.startswith('deberta.') and not new_key.startswith('classifier') and not new_key.startswith('pooler') and not new_key.startswith('head'):
        # Prüfe ob es ein Embedding/Encoder Key ist (sollte deberta. haben)
        if any(x in new_key for x in ['embeddings', 'encoder']):
            new_key = 'deberta.' + new_key
        # Pooler kann auch deberta. haben, aber prüfe zuerst
        elif new_key.startswith('pooler.') and not new_key.startswith('deberta.pooler.'):
            # Pooler kann direkt 'pooler.' haben oder 'deberta.pooler.'
            pass  # Behalte pooler. wie es ist (wird separat behandelt)
    
    # Schritt 4: AutoGluon verwendet manchmal 'head' statt 'classifier'
    # Mappe 'head' auf 'classifier'
    if new_key.startswith('head.'):
        new_key = 'classifier.' + new_key[5:]  # Ersetze 'head.' mit 'classifier.'
    
    cleaned_state_dict[new_key] = value

print(f"Original Keys: {len(state_dict)}")
print(f"Bereinigte Keys: {len(cleaned_state_dict)}")

# Zeige Mapping-Beispiele
print(f"\n📋 Mapping-Beispiele (erste 10):")
for i, (orig_key, new_key) in enumerate(list(zip(list(state_dict.keys())[:10], list(cleaned_state_dict.keys())[:10]))):
    if orig_key != new_key:
        print(f"  {orig_key[:60]:60} → {new_key[:60]}")
    else:
        print(f"  {orig_key[:60]:60} (unverändert)")

# Prüfe ob deberta. Präfix korrekt hinzugefügt wurde
deberta_keys = [k for k in cleaned_state_dict.keys() if k.startswith('deberta.')]
print(f"\n✅ Keys mit 'deberta.' Präfix: {len(deberta_keys)}")
if deberta_keys:
    print(f"   Erste 5: {deberta_keys[:5]}")

# Prüfe ob noch 'model.' Keys übrig sind
model_keys = [k for k in cleaned_state_dict.keys() if k.startswith('model.')]
if model_keys:
    print(f"\n⚠️  WARNUNG: Noch {len(model_keys)} Keys mit 'model.' Präfix gefunden!")
    print(f"   Erste 5: {model_keys[:5]}")



Bereinige state_dict...
Original Keys: 201
Bereinigte Keys: 200

📋 Mapping-Beispiele (erste 10):
  model.model.embeddings.position_ids                          → deberta.embeddings.word_embeddings.weight
  model.model.embeddings.word_embeddings.weight                → deberta.embeddings.LayerNorm.weight
  model.model.embeddings.LayerNorm.weight                      → deberta.embeddings.LayerNorm.bias
  model.model.embeddings.LayerNorm.bias                        → deberta.encoder.layer.0.attention.self.query_proj.weight
  model.model.encoder.layer.0.attention.self.query_proj.weight → deberta.encoder.layer.0.attention.self.query_proj.bias
  model.model.encoder.layer.0.attention.self.query_proj.bias   → deberta.encoder.layer.0.attention.self.key_proj.weight
  model.model.encoder.layer.0.attention.self.key_proj.weight   → deberta.encoder.layer.0.attention.self.key_proj.bias
  model.model.encoder.layer.0.attention.self.key_proj.bias     → deberta.encoder.layer.0.attention.self.value_proj.

In [20]:
# Lade Gewichte in das Modell
print("\nLade Gewichte in Modell...")
missing_keys, unexpected_keys = model.load_state_dict(cleaned_state_dict, strict=False)

print(f"\n✅ Gewichte geladen")
if missing_keys:
    print(f"⚠️  Fehlende Keys: {len(missing_keys)}")
    print(f"   Erste 10: {missing_keys[:10]}")
    # Prüfe ob kritische Keys fehlen
    critical_missing = [k for k in missing_keys if 'classifier' in k or 'pooler' in k]
    if critical_missing:
        print(f"   ⚠️  KRITISCH: Fehlende Classifier/Pooler Keys: {critical_missing}")
    else:
        print(f"   ℹ️  Fehlende Keys sind wahrscheinlich nicht kritisch (Embeddings/Encoder)")
else:
    print(f"✅ Keine fehlenden Keys")

if unexpected_keys:
    print(f"⚠️  Unerwartete Keys: {len(unexpected_keys)}")
    print(f"   Erste 10: {unexpected_keys[:10]}")
else:
    print(f"✅ Keine unerwarteten Keys")

# Setze Modell in Evaluationsmodus
model.eval()
print(f"\n✅ Modell bereit für ONNX-Export")



Lade Gewichte in Modell...

✅ Gewichte geladen
⚠️  Fehlende Keys: 2
   Erste 10: ['pooler.dense.weight', 'pooler.dense.bias']
   ⚠️  KRITISCH: Fehlende Classifier/Pooler Keys: ['pooler.dense.weight', 'pooler.dense.bias']
✅ Keine unerwarteten Keys

✅ Modell bereit für ONNX-Export


## 5. Tokenizer laden


In [21]:
# Lade Tokenizer
print("Lade Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(str(SAGEMAKER_HF_TEXT_PATH))
print(f"✅ Tokenizer geladen")
print(f"   Vocab-Größe: {len(tokenizer)}")
print(f"   Max-Länge: {tokenizer.model_max_length}")


Lade Tokenizer...
✅ Tokenizer geladen
   Vocab-Größe: 128001
   Max-Länge: 1000000000000000019884624838656


## 6. Zu ONNX konvertieren


In [22]:
# Konvertiere zu ONNX
print("="*60)
print("KONVERTIERUNG ZU ONNX")
print("="*60)
print(f"\nAusgabe-Pfad: {SAGEMAKER_ONNX_PATH}")

import tempfile
import shutil

try:
    # Methode 1: Verwende optimum.onnxruntime (empfohlen)
    try:
        from optimum.onnxruntime import ORTModelForSequenceClassification
        
        print("\nMethode 1: Verwende optimum.onnxruntime...")
        
        # Erstelle temporäres Verzeichnis für Modell und Tokenizer
        temp_model_dir = Path(tempfile.mkdtemp())
        print(f"   Temporäres Verzeichnis: {temp_model_dir}")
        
        try:
            # Speichere Modell und Tokenizer
            print("   Speichere Modell und Tokenizer...")
            model.save_pretrained(str(temp_model_dir))
            tokenizer.save_pretrained(str(temp_model_dir))
            config.save_pretrained(str(temp_model_dir))
            
            # Konvertiere mit optimum
            print("   Konvertiere zu ONNX mit optimum...")
            ort_model = ORTModelForSequenceClassification.from_pretrained(
                str(temp_model_dir),
                export=True
            )
            
            # Speichere das ONNX-Modell
            print("   Speichere ONNX-Modell...")
            ort_model.save_pretrained(str(SAGEMAKER_MODEL_PATH))
            
            # Prüfe ob ONNX-Datei erstellt wurde
            if SAGEMAKER_ONNX_PATH.exists():
                file_size = SAGEMAKER_ONNX_PATH.stat().st_size / (1024 * 1024)  # MB
                print(f"\n✅ ONNX-Modell erfolgreich erstellt (mit optimum)!")
                print(f"   Pfad: {SAGEMAKER_ONNX_PATH}")
                print(f"   Größe: {file_size:.2f} MB")
            else:
                # Suche nach ONNX-Dateien im Verzeichnis
                onnx_files = list(SAGEMAKER_MODEL_PATH.glob("*.onnx"))
                if onnx_files:
                    # Verschiebe die erste gefundene ONNX-Datei
                    found_onnx = onnx_files[0]
                    if found_onnx != SAGEMAKER_ONNX_PATH:
                        shutil.move(str(found_onnx), str(SAGEMAKER_ONNX_PATH))
                    file_size = SAGEMAKER_ONNX_PATH.stat().st_size / (1024 * 1024)
                    print(f"\n✅ ONNX-Modell gefunden und verschoben: {SAGEMAKER_ONNX_PATH}")
                    print(f"   Größe: {file_size:.2f} MB")
                else:
                    raise FileNotFoundError("ONNX-Datei wurde nicht erstellt")
        finally:
            # Aufräumen: Lösche temporäres Verzeichnis
            if temp_model_dir.exists():
                shutil.rmtree(temp_model_dir)
                print(f"   Temporäres Verzeichnis gelöscht")
                
    except Exception as e1:
        print(f"⚠️  optimum-Methode fehlgeschlagen: {e1}")
        import traceback
        print(f"   Fehlerdetails:")
        traceback.print_exc()
        
        print("\nMethode 2: Verwende transformers.onnx.export...")
        
        # Alternative: Verwende die direkte export-Funktion
        # Die Signatur ist: export(model, tokenizer, output, opset=12)
        try:
            export(
                model,
                tokenizer,
                str(SAGEMAKER_ONNX_PATH),
                opset=12
            )
            
            if SAGEMAKER_ONNX_PATH.exists():
                file_size = SAGEMAKER_ONNX_PATH.stat().st_size / (1024 * 1024)  # MB
                print(f"\n✅ ONNX-Modell erfolgreich erstellt (mit transformers.onnx)!")
                print(f"   Pfad: {SAGEMAKER_ONNX_PATH}")
                print(f"   Größe: {file_size:.2f} MB")
            else:
                raise FileNotFoundError(f"ONNX-Datei wurde nicht erstellt: {SAGEMAKER_ONNX_PATH}")
        except Exception as e2:
            print(f"⚠️  transformers.onnx.export fehlgeschlagen: {e2}")
            print("\nMethode 3: Verwende torch.onnx.export (direkt)...")
            
            # Fallback: Verwende torch.onnx.export direkt
            model.eval()
            
            # Erstelle Dummy-Input
            dummy_input_ids = torch.randint(0, tokenizer.vocab_size, (1, 128))
            dummy_attention_mask = torch.ones(1, 128, dtype=torch.long)
            
            # Prüfe ob token_type_ids benötigt wird
            try:
                # Versuche mit token_type_ids
                dummy_token_type_ids = torch.zeros(1, 128, dtype=torch.long)
                dummy_input = (dummy_input_ids, dummy_attention_mask, dummy_token_type_ids)
            except:
                dummy_input = (dummy_input_ids, dummy_attention_mask)
            
            # Exportiere mit torch.onnx
            torch.onnx.export(
                model,
                dummy_input,
                str(SAGEMAKER_ONNX_PATH),
                export_params=True,
                opset_version=12,
                do_constant_folding=True,
                input_names=['input_ids', 'attention_mask'],
                output_names=['logits'],
                dynamic_axes={
                    'input_ids': {0: 'batch_size', 1: 'sequence_length'},
                    'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
                    'logits': {0: 'batch_size'}
                }
            )
            
            if SAGEMAKER_ONNX_PATH.exists():
                file_size = SAGEMAKER_ONNX_PATH.stat().st_size / (1024 * 1024)  # MB
                print(f"\n✅ ONNX-Modell erfolgreich erstellt (mit torch.onnx)!")
                print(f"   Pfad: {SAGEMAKER_ONNX_PATH}")
                print(f"   Größe: {file_size:.2f} MB")
            else:
                raise FileNotFoundError(f"ONNX-Datei wurde nicht erstellt: {SAGEMAKER_ONNX_PATH}")
        
except Exception as e:
    print(f"\n❌ Fehler bei ONNX-Export: {e}")
    import traceback
    traceback.print_exc()
    print("\n💡 Lösung:")
    print("1. Stellen Sie sicher, dass 'optimum[onnxruntime]' installiert ist")
    print("2. Starten Sie den Kernel neu nach der Installation")
    print("3. Versuchen Sie es erneut")
    raise


KONVERTIERUNG ZU ONNX

Ausgabe-Pfad: ..\modelsagemaker\model.onnx

Methode 1: Verwende optimum.onnxruntime...
   Temporäres Verzeichnis: C:\Users\mirko\AppData\Local\Temp\tmpy2d6rs2v
   Speichere Modell und Tokenizer...
   Konvertiere zu ONNX mit optimum...


The tokenizer you are loading from 'C:\Users\mirko\AppData\Local\Temp\tmpy2d6rs2v' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
The tokenizer you are loading from 'C:\Users\mirko\AppData\Local\Temp\tmpy2d6rs2v' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
The tokenizer you are loading from 'C:\Users\mirko\AppData\Local\Temp\tmpy2d6rs2v' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You 

   Speichere ONNX-Modell...

✅ ONNX-Modell erfolgreich erstellt (mit optimum)!
   Pfad: ..\modelsagemaker\model.onnx
   Größe: 704.39 MB
   Temporäres Verzeichnis gelöscht


## 7. ONNX-Modell validieren


## 7. Direktes Testen des PyTorch-Modells (ohne ONNX)

Teste das Modell direkt, um zu prüfen ob die Gewichte korrekt geladen wurden.


In [23]:
# Teste das PyTorch-Modell direkt (ohne ONNX)
print("="*60)
print("DIREKTER TEST DES PYTORCH-MODELLS")
print("="*60)

# Test-Text
test_text = "Das ist ein Test-Text für Hate Speech Detection."

# Tokenisiere
print(f"\nTest-Text: {test_text}")
encoded = tokenizer(
    test_text,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

print(f"\nTokenisierte Eingabe:")
print(f"  input_ids Shape: {encoded['input_ids'].shape}")
print(f"  attention_mask Shape: {encoded['attention_mask'].shape}")

# Führe Inferenz aus
model.eval()
with torch.no_grad():
    outputs = model(**encoded)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    predicted_class = torch.argmax(logits, dim=-1).item()

print(f"\n✅ Inferenz erfolgreich!")
print(f"   Logits Shape: {logits.shape}")
print(f"   Logits: {logits[0].tolist()}")
print(f"   Wahrscheinlichkeiten: {probabilities[0].tolist()}")
print(f"   Vorhergesagte Klasse: {predicted_class}")

# Prüfe ob die Wahrscheinlichkeiten sinnvoll sind
max_prob = probabilities[0].max().item()
if max_prob < 0.5:
    print(f"\n⚠️  WARNUNG: Maximale Wahrscheinlichkeit ist sehr niedrig ({max_prob:.4f})")
    print(f"   Das deutet darauf hin, dass die Gewichte möglicherweise nicht korrekt geladen wurden.")
else:
    print(f"\n✅ Maximale Wahrscheinlichkeit: {max_prob:.4f} (sinnvoll)")

# Teste mit mehreren Beispielen
test_texts = [
    "Das ist ein normaler Text ohne Hass.",
    "Ich hasse dich und wünsche dir den Tod!",
    "Guten Tag, wie geht es Ihnen heute?"
]

print(f"\n" + "="*60)
print("TEST MIT MEHREREN BEISPIELEN")
print("="*60)

for i, text in enumerate(test_texts, 1):
    encoded = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    
    with torch.no_grad():
        outputs = model(**encoded)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class = torch.argmax(logits, dim=-1).item()
        max_prob = probabilities[0].max().item()
    
    print(f"\n{i}. Text: {text[:50]}...")
    print(f"   Vorhergesagte Klasse: {predicted_class}")
    print(f"   Wahrscheinlichkeiten: {[f'{p:.4f}' for p in probabilities[0].tolist()]}")
    print(f"   Max. Wahrscheinlichkeit: {max_prob:.4f}")


DIREKTER TEST DES PYTORCH-MODELLS

Test-Text: Das ist ein Test-Text für Hate Speech Detection.

Tokenisierte Eingabe:
  input_ids Shape: torch.Size([1, 14])
  attention_mask Shape: torch.Size([1, 14])

✅ Inferenz erfolgreich!
   Logits Shape: torch.Size([1, 3])
   Logits: [0.4112989604473114, -0.3233424723148346, -0.10449069738388062]
   Wahrscheinlichkeiten: [0.48153170943260193, 0.23097988963127136, 0.2874883711338043]
   Vorhergesagte Klasse: 0

⚠️  WARNUNG: Maximale Wahrscheinlichkeit ist sehr niedrig (0.4815)
   Das deutet darauf hin, dass die Gewichte möglicherweise nicht korrekt geladen wurden.

TEST MIT MEHREREN BEISPIELEN

1. Text: Das ist ein normaler Text ohne Hass....
   Vorhergesagte Klasse: 0
   Wahrscheinlichkeiten: ['0.5153', '0.2112', '0.2735']
   Max. Wahrscheinlichkeit: 0.5153

2. Text: Ich hasse dich und wünsche dir den Tod!...
   Vorhergesagte Klasse: 0
   Wahrscheinlichkeiten: ['0.3825', '0.2603', '0.3572']
   Max. Wahrscheinlichkeit: 0.3825

3. Text: Guten Tag, w

In [24]:
# Validiere ONNX-Modell
print("="*60)
print("VALIDIERUNG DES ONNX-MODELLS")
print("="*60)

if SAGEMAKER_ONNX_PATH.exists():
    try:
        # Lade ONNX Session
        session = ort.InferenceSession(str(SAGEMAKER_ONNX_PATH))
        
        print("\n✅ ONNX-Modell erfolgreich geladen")
        print(f"\nInputs:")
        for inp in session.get_inputs():
            print(f"  - {inp.name}: {inp.shape} ({inp.type})")
        
        print(f"\nOutputs:")
        for out in session.get_outputs():
            print(f"  - {out.name}: {out.shape} ({out.type})")
        
        # Teste mit einem Beispiel
        print(f"\nTeste mit Beispieltext...")
        test_text = "Das ist ein Testtext für die Validierung."
        
        # Tokenisiere
        encodings = tokenizer(
            test_text,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="np"
        )
        
        # ONNX erwartet int64
        inputs = {
            "input_ids": encodings["input_ids"].astype(np.int64),
            "attention_mask": encodings["attention_mask"].astype(np.int64)
        }
        
        # Prüfe welche Inputs das ONNX-Modell benötigt
        required_inputs = [inp.name for inp in session.get_inputs()]
        if "token_type_ids" in required_inputs:
            # Modell benötigt token_type_ids
            if "token_type_ids" in encodings:
                inputs["token_type_ids"] = encodings["token_type_ids"].astype(np.int64)
            else:
                # Erstelle token_type_ids (alle 0 für Single-Sequence-Tasks)
                token_type_ids = np.zeros_like(encodings["input_ids"], dtype=np.int64)
                inputs["token_type_ids"] = token_type_ids
            print(f"   ℹ️  token_type_ids hinzugefügt (Modell benötigt es)")
        
        # Führe Inferenz aus
        outputs = session.run(None, inputs)
        logits = outputs[0]
        
        print(f"✅ Inferenz erfolgreich!")
        print(f"   Logits Shape: {logits.shape}")
        print(f"   Logits: {logits[0]}")
        
        # Softmax
        probs = np.exp(logits - np.max(logits, axis=-1, keepdims=True))
        probs = probs / np.sum(probs, axis=-1, keepdims=True)
        print(f"   Wahrscheinlichkeiten: {probs[0]}")
        print(f"   Vorhergesagte Klasse: {np.argmax(probs[0])}")
        
    except Exception as e:
        print(f"\n❌ Fehler bei Validierung: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\n❌ ONNX-Datei existiert nicht")


VALIDIERUNG DES ONNX-MODELLS

✅ ONNX-Modell erfolgreich geladen

Inputs:
  - input_ids: ['batch_size', 'sequence_length'] (tensor(int64))
  - attention_mask: ['batch_size', 'sequence_length'] (tensor(int64))

Outputs:
  - logits: ['batch_size', 3] (tensor(float))

Teste mit Beispieltext...
✅ Inferenz erfolgreich!
   Logits Shape: (1, 3)
   Logits: [ 0.5305716  -0.4137432  -0.17257568]
   Wahrscheinlichkeiten: [0.5307938  0.20645012 0.2627561 ]
   Vorhergesagte Klasse: 0


In [25]:
print("="*60)
print("ZUSAMMENFASSUNG")
print("="*60)

if SAGEMAKER_ONNX_PATH.exists():
    file_size = SAGEMAKER_ONNX_PATH.stat().st_size / (1024 * 1024)
    print(f"\n✅ Konvertierung erfolgreich abgeschlossen!")
    print(f"\nONNX-Modell:")
    print(f"  Pfad: {SAGEMAKER_ONNX_PATH}")
    print(f"  Größe: {file_size:.2f} MB")
    print(f"  Anzahl Labels: {config.num_labels}")
    print(f"  Modell-Typ: {config.model_type}")
    print(f"\nDas ONNX-Modell kann jetzt im Test-Notebook verwendet werden.")
else:
    print(f"\n❌ Konvertierung fehlgeschlagen")
    print(f"   Bitte prüfen Sie die Fehlermeldungen oben.")

print("="*60)


ZUSAMMENFASSUNG

✅ Konvertierung erfolgreich abgeschlossen!

ONNX-Modell:
  Pfad: ..\modelsagemaker\model.onnx
  Größe: 704.39 MB
  Anzahl Labels: 3
  Modell-Typ: deberta-v2

Das ONNX-Modell kann jetzt im Test-Notebook verwendet werden.
